<a href="https://colab.research.google.com/github/wildlifeai/pepeketua_zooniverse/blob/main/frog_zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the scripts to upload photos of Archey's frogs to a Zooniverse project, where researchers label landmarks in the frogs to train ML algorithms.

# Download frog photos

###Add shortcuts to the compressed photos

To download the photos of the frogs into this Google Colab you first need to add shortcuts in your Google drive to the [five zipped folders](https://drive.google.com/file/d/1XXSrATFX1l-J0CUE4m6UfoOBp9zv3XOr/view?usp=sharing) with the photos. 

To add the shortcuts:
* go to the "Shared with me" section in your Google drive,
* find the five zipped folders,
* click on "Add shorcut to Drive" and
* save the shortcuts (we created a folder called "frog_photos" and saved them there).

*Specify* the folder in your Google drive where you saved the shortcuts to the photos (in our case "frog_photos").

In [ ]:
dir_shortcuts = "/content/drive/My Drive/frog_photos/"

*If you can't access the five zipped folders please [email Victor](victor@wildlife.ai). 

###Download the photos

To download and unzip the five folders with the photos you will need to grant access to Google file stream. 

The photos will be temporarily stored in your Google Colab disk and will not count towards your Google drive storage.

*Uncomment lines to unzip all five folders (it might take a while though).

In [2]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

whareorino_a = zipfile.ZipFile(dir_shortcuts + "whareorino_a.zip", 'r')
whareorino_a.extractall("/tmp")
whareorino_a.close()

# whareorino_b = zipfile.ZipFile(dir_shortcuts + "whareorino_b.zip", 'r')
# whareorino_b.extractall("/tmp")
# whareorino_b.close()

# whareorino_c = zipfile.ZipFile(dir_shortcuts + "whareorino_c.zip", 'r')
# whareorino_c.extractall("/tmp")
# whareorino_c.close()

# whareorino_d = zipfile.ZipFile(dir_shortcuts + "whareorino_d.zip", 'r')
# whareorino_d.extractall("/tmp")
# whareorino_d.close()

# pukeokahu = zipfile.ZipFile(dir_shortcuts + "pukeokahu.zip", 'r')
# pukeokahu.extractall("/tmp")
# pukeokahu.close()

Mounted at /content/drive


In [3]:
ls

'Colab Notebooks'/   frog_photos/             Research/
 Communication/     'Funding applications'/  'Rob and Sally 18 sept.gdoc'
 Education/          Governance/             'Untitled document.gdoc'
 Evan_chat.gdoc     'Rachel Selwyn.gdoc'      Victor/
